### import 및 함수

In [1]:
# dict_kcs = {"kcs" : "pullPath"}
# 20494491752A40E088201 /volume1/솔림헬프/기타/전북-01/20494491/20494491_보증인 오병석_기타_2022본11747_유체접수증.pdf
import os
import pickle
import binascii
import re
from os.path import join

def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

def get_kcs(path_192) :
    crc32 = str(crc32_checksum(path_192))
    size = str(os.path.getsize(path_192))
    f = os.path.split(path_192)[1]
    key = f[:8]
    kcs = key + crc32 + size 
    return kcs

def read_kcs(path_kcs) :
    with open(path_kcs, "rb") as pkl :
        return pickle.load(pkl)

def delete_kcs(filePath, dict_kcs) :
    print("삭제 전 : ", len(dict_kcs))
    k = os.path.split(filePath)[1][:8]
    c = str(crc32_checksum(filePath))
    s = str(os.path.getsize(filePath))
    kcs = k + c + s
    if kcs in dict_kcs :
        print(f"kcs : {kcs}   path : {dict_kcs[kcs]}")
        del dict_kcs[kcs]
        print("삭제하였습니다.")
    else :
      print(filePath, kcs, "없는 키값")
    print("삭제 후 : ", len(dict_kcs))

def write_kcs(path_kcs, dict_kcs) :
    with open(path_kcs, "wb") as pkl :
        pickle.dump(dict_kcs, pkl)

# def update_kcs(src, dst, dict_kcs) :
    
#     kcs = get_kcs(src)
#     print(dict_kcs[kcs])

#     new_dst = re.sub(r"\\\\192.168.0.75", "/volume1", dst)
#     new_dst = re.sub(r"\\", "/", new_dst)
#     dict_kcs[kcs] = new_dst
#     print(dict_kcs[kcs], "dict수정 성공")

def update_kcs(src, dst, dict_kcs) :
    # 채무자키
    src_key = os.path.split(src)[1][:8]
    dst_key = os.path.split(dst)[1][:8]
    # kcs
    kcs = get_kcs(src)
    before_value = dict_kcs[kcs] # 수정되기 전에 변수에 담아두어야
    # 윈도우 path를 리눅스 path로
    new_dst = re.sub(r"\\\\192.168.0.75", "/volume1", dst)
    new_dst = re.sub(r"\\", "/", new_dst)

    # 채무자키가 달라졌을 때 > dict의 key를 수정해야 하므로 삭제 후 새값 추가
    if src_key != dst_key :
        new_kcs = dst_key + kcs[8:]
        del dict_kcs[kcs] # 구값 삭제
    else : new_kcs = kcs
    
    # 새값 추가/수정(채무자키 동일했다면 kcs == new_kcs 따라서 수정된다.)
    dict_kcs[new_kcs] = new_dst

    print(f"dict 키 변화 : {kcs} >>> {new_kcs}")
    print(f"dict 값 변화 : {before_value} >>> {dict_kcs[new_kcs]}")
    

path_kcs = r"\\192.168.0.75\스캔파일\스캔파일log\_project\대성파일\중복조사\kcs별 파일정보_대성.pkl"

### dict load

In [2]:
# 읽기
dict_kcs = read_kcs(path_kcs)
print("최초dict_kcs개수 :", len(dict_kcs))

최초dict_kcs개수 : 11285


In [3]:
dict_kcs["10622626DA6BF605302827"]

'/volume1/대성/3.집행권원/씨에프-01/10622626/10622626_서정식_집행권원.pdf'

### dict삭제 > 파일은 직접, 파일명 변경후 재업하면 수정작업과 동일

In [3]:
# 1) 단일파일 dict 삭제
####################################
# 에브리띵에서 전체경로복사하면 편함, 채무자키만 유지하면 파일위치, 파일명는 상관없음
path_192 = r"\\192.168.0.75\솔림헬프\3.집행권원\더키움\20426946\20426946_강호정_집행권원.pdf"
####################################
delete_kcs(path_192, dict_kcs)

삭제 전 :  401379
kcs : 2042694633A289931864652   path : /volume1/솔림헬프/3.집행권원/더키움/20426946/20426946_강호정_집행권원.pdf
삭제하였습니다.
삭제 후 :  401378


In [6]:
# dict 파일로 저장
write_kcs(path_kcs, dict_kcs)

In [18]:
# 2) 파일 삭제/이동
# 관리제외 폴더로 이동
import shutil
src = path_192
dst_d = join(r"\\192.168.0.75\삭제예정파일\관리제외", src.split(os.path.sep)[-3], src.split(os.path.sep)[-2])
if not os.path.exists(dst_d) : os.makedirs(dst_d)
dst = join(dst_d, src.split(os.path.sep)[-1])
shutil.move(src, dst)

'\\\\192.168.0.75\\삭제예정파일\\관리제외\\아인스\\20410937\\20410937_박효동_개인회생_2019개회12743.pdf'

In [4]:
# 폴더 내 전체 파일 dict 삭제
path = r"\\192.168.0.75\스캔파일\대성 스캔파일 업로드\구스캔파일"
for f in os.listdir(path) :
    if os.path.isfile(f) : 
        path_192 = os.path.join(path, f)
        delete_kcs(path_192, dict_kcs)


삭제 전 :  11285
kcs : 106226230ED002BB141419   path : /volume1/대성/1.원인서류/백운-01/10622623/10622623_성해숙_원인서류_이정옥.pdf
삭제하였습니다.
삭제 후 :  11284
삭제 전 :  11284
kcs : 106226722674D54D163992   path : /volume1/대성/1.원인서류/백운-01/10622672/10622672_정수강_원인서류_박상훈.pdf
삭제하였습니다.
삭제 후 :  11283
삭제 전 :  11283
kcs : 106228835599EA24170723   path : /volume1/대성/1.원인서류/백운-01/10622883/10622883_김남이_원인서류_소영춘.pdf
삭제하였습니다.
삭제 후 :  11282
삭제 전 :  11282
kcs : 1062288590C38AA4133744   path : /volume1/대성/1.원인서류/백운-01/10622885/10622885_김보영_원인서류_최낙동.pdf
삭제하였습니다.
삭제 후 :  11281
삭제 전 :  11281
kcs : 10622893312EC34E162012   path : /volume1/대성/1.원인서류/백운-01/10622893/10622893_박세홍_원인서류_김정숙.pdf
삭제하였습니다.
삭제 후 :  11280
삭제 전 :  11280
kcs : 106228966D756D6C170045   path : /volume1/대성/1.원인서류/백운-01/10622896/10622896_박연희_원인서류_박기덕.pdf
삭제하였습니다.
삭제 후 :  11279
삭제 전 :  11279
kcs : 10622896BC602F42135993   path : /volume1/대성/1.원인서류/백운-01/10622896/10622896_박연희_원인서류_정선주.pdf
삭제하였습니다.
삭제 후 :  11278
삭제 전 :  11278
kcs : 10622897C7CF5435133112   path : /vo

PermissionError: [Errno 13] Permission denied: '\\\\192.168.0.75\\스캔파일\\대성 스캔파일 업로드\\구스캔파일\\파일명에러'

### dict 수정 
- 폴더 달라질 때는 그냥 새로 업로드하는게 낫겠다.

In [5]:
########################################################### 20514967
src = r"\\192.168.0.75\솔림헬프\3.집행권원\DNP-01\20414371\20414371_조영민_집행권원.pdf"
dst = r"\\192.168.0.75\솔림헬프\3.집행권원\DNP-01\20414371\20414371 조영민 2023차전102996.pdf"
###########################################################

# 1) dict 수정
update_kcs(src, dst, dict_kcs)

dict 키 변화 : 20414371ECD2146F176769 >>> 20414371ECD2146F176769
dict 값 변화 : /volume1/솔림헬프/3.집행권원/DNP-01/20414371/20414371_조영민_집행권원.pdf >>> /volume1/솔림헬프/3.집행권원/DNP-01/20414371/20414371 조영민 2023차전102996.pdf


In [6]:
# 2) 파일수정 : dict보다 먼저 하면 kcs를 읽을 수가 없음
os.rename(src, dst)
print("파일명(경로) 수정 성공")

파일명(경로) 수정 성공


In [7]:
# 3) dict 파일로 저장 # 약 20초 소요
write_kcs(path_kcs, dict_kcs)

#### 폴더 내 여러 파일 

In [12]:
########################################################
path = r"\\192.168.0.75\솔림헬프\5.등초본\한울가람\20419713"
for f in os.listdir(path) :
    src = join(path, f)
    dst_file = re.sub("염직", "88염직", f)
    ####################################
    dst = join(path, dst_file)
    if src != dst :
        update_kcs(src, dst, dict_kcs)
        os.rename(src, dst)

dict 키 변화 : 20419713364E4B38368410 >>> 20419713364E4B38368410
dict 값 변화 : /volume1/솔림헬프/5.등초본/한울가람/20419713/20419713_염직 보증인 강태수_초본_220926.pdf >>> /volume1/솔림헬프/5.등초본/한울가람/20419713/20419713_88염직 보증인 강태수_초본_220926.pdf


In [14]:
write_kcs(path_kcs, dict_kcs)